In [2]:
import pandas as pd
import numpy as np

In [16]:
respiratory=pd.read_csv("respiratoryCharting.csv", chunksize=10000)

In [17]:
l_r=[]
for chunk in respiratory:
    l_r.append(chunk)    

In [18]:
len(l_r)

2017

In [19]:
df_resp=pd.concat(l_r, sort=False)

In [20]:
df_resp.head()

,respchartid,patientunitstayid,respchartoffset,respchartentryoffset,respcharttypecat,respchartvaluelabel,respchartvalue
0,3,221468,4679,4679,respFlowCareData,RT Vent On/Off,Continued
1,4,221468,4679,4679,respFlowSettings,FiO2,40
2,5,221468,4679,4679,respFlowSettings,Vent Rate,14
3,6,221468,4679,4679,respFlowSettings,Tidal Volume (set),450
4,7,221468,4679,4679,respFlowSettings,TV/kg IBW,5.4720


In [27]:
respFiO2=df_resp.loc[df_resp['respchartvaluelabel']=='FiO2']

In [28]:
respFiO2.head()

,respchartid,patientunitstayid,respchartoffset,respchartentryoffset,respcharttypecat,respchartvaluelabel,respchartvalue
1,4,221468,4679,4679,respFlowSettings,FiO2,40
15,60,152518,27557,27557,respFlowSettings,FiO2,50
22,81,204319,1917,1917,respFlowSettings,FiO2,60
24,85,241257,4468,4468,respFlowSettings,FiO2,70
26,105,192236,27137,27137,respFlowSettings,FiO2,40


In [29]:
respFiO2.loc[respFiO2['respchartvalue']==0, 'respchartvalue']=1

In [30]:
respFiO2['respchartvalue']=respFiO2['respchartvalue'].str.replace('%','')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
respFiO2=respFiO2[['patientunitstayid','respchartentryoffset', 'respchartvaluelabel', 'respchartvalue']]

In [42]:
respFiO2.dtypes

patientunitstayid         int32
respchartentryoffset      int32
respchartvaluelabel      object
respchartvalue          float32
dtype: object

In [39]:
respFiO2[['patientunitstayid','respchartentryoffset']]=respFiO2[['patientunitstayid','respchartentryoffset']].astype('int32')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [41]:
respFiO2[['respchartvalue']]=respFiO2[['respchartvalue']].astype('str').astype('float32')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [43]:
df_lab=pd.read_csv("labs_before_FiO2.csv")
df_lab['paO2_FiO2']=np.nan

In [44]:
df_lab.head()

,patientunitstayid,labresultoffset,labname,labresult,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset,paO2_FiO2
0,141168,1701,urinary creatinine,173.12,mg/dL,mg/dL,1757,NaN
1,141168,2026,paO2,68.00,mm Hg,mm Hg,2054,NaN
2,141168,2141,paO2,121.00,mm Hg,mm Hg,2155,NaN
3,141168,1133,platelets x 1000,213.00,K/mcL,K/mcL,1196,NaN
4,141168,2010,paO2,42.00,mm Hg,mm Hg,2026,NaN


In [45]:
df_lab=df_lab[['patientunitstayid', 'labresultoffset', 'labname', 'labresult', 'paO2_FiO2' ]]

In [46]:
df_lab.dtypes

patientunitstayid      int64
labresultoffset        int64
labname               object
labresult            float64
paO2_FiO2            float64
dtype: object

In [47]:
df_lab[['patientunitstayid','labresultoffset']]=df_lab[['patientunitstayid','labresultoffset']].astype('int32')

In [48]:
df_lab[['labresult','paO2_FiO2']]=df_lab[['labresult','paO2_FiO2']].astype('float32')

<h1>WARNING! The next block of code takes about 15 hours to run. Please keep that in mind. 
If the processed csv is needed, please contact me</h1>

In [50]:
count=0
for row in df_lab.itertuples():
    if (row.labname=='paO2'):
        pid=row.patientunitstayid
        toffset=row.labresultoffset
        index=row.Index
        if(any(respFiO2.loc[(respFiO2['patientunitstayid']==pid) & ((respFiO2['respchartentryoffset']<=toffset+200) & (respFiO2['respchartentryoffset']>=toffset-200)),'respchartvalue'].values)):
            result = row.labresult/(respFiO2.loc[(respFiO2['patientunitstayid']==pid) & ((respFiO2['respchartentryoffset']<=toffset+200) & (respFiO2['respchartentryoffset']>=toffset-200)),'respchartvalue'].iloc[0]/100)
            df_lab['paO2_FiO2'][index]=result      
        else:
            df_lab['paO2_FiO2'][index]=df_lab['paO2_FiO2'].mean()
    count+=1
    if((count%10000)==0):
        print(count)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


10000
20000
30000


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  


40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390000
1400000
141000

In [ ]:
for pid in df_lab['patientunitstayid'].unique():
    df_lab.loc['']
    

In [ ]:
df_lab.head()

In [58]:
respFiO2.loc[(respFiO2['patientunitstayid']==147037) & ((respFiO2['respchartentryoffset']<=4100+200) & (respFiO2['respchartentryoffset']>=4100-200)),'respchartvalue']

Series([], Name: respchartvalue, dtype: object)

In [67]:
df_lab.iloc[750]

patientunitstayid              147037
labresultoffset                 -1975
labname              platelets x 1000
labresult                         185
paO2_FiO2                            
Name: 7570, dtype: object

In [69]:
len(df_lab.loc[df_lab['labname']=='paO2'])

420762

In [72]:
np.ones((len(df_lab),1))

numpy.ndarray

In [107]:
df_lab['labresult'].mean()

151.67764282226562

In [104]:
arr.append(2)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [51]:
import csv

In [1]:
df_lab.to_csv("labs_after_FiO2.csv", sep=',', index=False, encoding='utf-8')

NameError: name 'df_lab' is not defined

<h1>Continue from here</h1>

In [17]:
pre_final_lab=pd.read_csv("labs_after_FiO2.csv")

In [18]:
pre_final_lab['platelets_x_1000']=np.nan
pre_final_lab['total_bilirubin']=np.nan
pre_final_lab['urinary_creatinine']=np.nan

In [19]:
pre_final_lab['platelets_x_1000']=pre_final_lab.loc[pre_final_lab['labname']=='platelets x 1000','labresult']
pre_final_lab['total_bilirubin']=pre_final_lab.loc[pre_final_lab['labname']=='total bilirubin','labresult']
pre_final_lab['urinary_creatinine']=pre_final_lab.loc[pre_final_lab['labname']=='urinary creatinine','labresult']

In [20]:
pre_final_lab

,patientunitstayid,labresultoffset,labname,labresult,paO2_FiO2,platelets_x_1000,total_bilirubin,urinary_creatinine
0,141168,1701,urinary creatinine,173.12,NaN,NaN,NaN,173.12
1,141168,2026,paO2,68.00,136.000000,NaN,NaN,NaN
2,141168,2141,paO2,121.00,121.000000,NaN,NaN,NaN
3,141168,1133,platelets x 1000,213.00,NaN,213.0,NaN,NaN
4,141168,2010,paO2,42.00,84.000000,NaN,NaN,NaN
5,141168,2026,total bilirubin,5.20,NaN,NaN,5.2,NaN
6,141168,2026,platelets x 1000,187.00,NaN,187.0,NaN,NaN
7,141168,1805,paO2,41.00,82.000000,NaN,NaN,NaN
8,141168,516,total bilirubin,2.60,NaN,NaN,2.6,NaN
9,141168,1133,total bilirubin,4.10,NaN,NaN,4.1,NaN


In [21]:
pre_final_lab=pre_final_lab.replace([np.inf, -np.inf], np.nan)

In [22]:
del pre_final_lab['labresult']

In [26]:
pre_final_lab

,patientunitstayid,labresultoffset,labname,paO2_FiO2,platelets_x_1000,total_bilirubin,urinary_creatinine
0,141168,1701,urinary creatinine,NaN,NaN,NaN,173.12
1,141168,2026,paO2,136.000000,NaN,NaN,173.12
2,141168,2141,paO2,121.000000,NaN,NaN,173.12
3,141168,1133,platelets x 1000,121.000000,213.0,NaN,173.12
4,141168,2010,paO2,84.000000,213.0,NaN,173.12
5,141168,2026,total bilirubin,84.000000,213.0,5.2,173.12
6,141168,2026,platelets x 1000,84.000000,187.0,5.2,NaN
7,141168,1805,paO2,82.000000,187.0,5.2,NaN
8,141168,516,total bilirubin,82.000000,187.0,2.6,NaN
9,141168,1133,total bilirubin,82.000000,187.0,4.1,NaN


In [24]:
pre_final_lab.describe()

,patientunitstayid,labresultoffset,paO2_FiO2,platelets_x_1000,total_bilirubin,urinary_creatinine
count,2.010839e+06,2.010839e+06,190301.000000,1.139619e+06,425009.000000,13227.000000
mean,1.753460e+06,5.185645e+03,388.943193,2.157523e+02,1.558249,99.160934
std,1.003857e+06,3.606784e+04,2225.146286,1.208963e+02,3.586318,71.979411
min,1.411680e+05,-4.731957e+07,-212.500000,0.000000e+00,0.000000,0.400000
25%,9.594120e+05,1.070000e+02,137.500000,1.370000e+02,0.400000,48.000000
50%,1.644075e+06,2.125000e+03,214.285720,1.970000e+02,0.600000,82.840000
75%,2.770721e+06,6.822000e+03,316.666660,2.700000e+02,1.200000,130.555000
max,3.353263e+06,8.305830e+05,129666.664000,3.092000e+03,99.000000,1288.000000


<p>The following block is made to iterate over the entire dataset and plug missing values with both ffill and bfill techniques,
but filling 3 at a time. Due to the code taking a lot of time, I have left it as of now. I'll look into it further after the rest of
the tables are processed.<p>

In [ ]:
'''while(any(pre_final_lab['paO2_FiO2'].isna()) or any(pre_final_lab['platelets_x_1000'].isna()) or any(pre_final_lab['total_bilirubin'].isna()) or any(pre_final_lab['urinary_creatinine'].isna())):
    pre_final_lab['paO2_FiO2']=pre_final_lab['paO2_FiO2'].ffill(limit=3)
    pre_final_lab['platelets_x_1000']=pre_final_lab['platelets_x_1000'].ffill(limit=3)
    pre_final_lab['total_bilirubin']=pre_final_lab['total_bilirubin'].ffill(limit=3)
    pre_final_lab['urinary_creatinine']=pre_final_lab['urinary_creatinine'].ffill(limit=3)
    pre_final_lab['paO2_FiO2']=pre_final_lab['paO2_FiO2'].bfill(limit=3)
    pre_final_lab['platelets_x_1000']=pre_final_lab['platelets_x_1000'].bfill(limit=3)
    pre_final_lab['total_bilirubin']=pre_final_lab['total_bilirubin'].bfill(limit=3)
    pre_final_lab['urinary_creatinine']=pre_final_lab['urinary_creatinine'].bfill(limit=3)
'''
